In [2]:
!pip install yfinance
!pip install yahoofinancials


  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23447 sha256=9bfb6d0a72bf590f96ad3db0b0b9fc668c25970180d484e84b70e522558943e7
  Stored in directory: c:\users\bhargavi\appdata\local\pip\cache\wheels\7e\d5\f4\f628965651f61ae35c2bdc73571ce1a2026e1550a336b5ad4b
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8372 sha256=84c1bfd4c181a402b9f0f1c96d17fce56b63cbd2c4c9949027cdf74e82a0814e
  Stored in directory: c:\users\bhargavi\appdata\local\pip\cache\wheels\57\6d\a3\a39b839cc75274d2acfb1c58bfead2f726c6577fe8c4723f13
Successfully built yfinance multitasking
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15195 sha256=f8e65e38917be8d4a70e3c6bcbd93453a5bab1693939392abf266f85a5eaaf2f
  Stored in directory: c:\users\bhargavi\appdata\local\pip\cache\wheels\6a\90\0c\08c7ac2ce60b9ac91529417d471e59244f9f96848c86f14809
Successfully built yahoofinancials


In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import StandardScaler
import yfinance as yf
from yahoofinancials import YahooFinancials

In [53]:
#CREATING THE DATAFRAME TO STORE DATA
df = yf.download('DOGE-USD',
         start='2020-01-01',
         end='2021-05-07',
         progress=False)

In [54]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-31,0.002056,0.002056,0.002022,0.002028,0.002028,69469785
2020-01-01,0.002028,0.002052,0.002021,0.002033,0.002033,51180941
2020-01-02,0.002034,0.002110,0.001983,0.002009,0.002009,65071106
2020-01-03,0.002008,0.002177,0.001991,0.002145,0.002145,62619988
2020-01-04,0.002144,0.002490,0.002050,0.002241,0.002241,94227582
...,...,...,...,...,...,...
2021-05-02,0.391087,0.400029,0.364874,0.376046,0.376046,8040406918
2021-05-03,0.373631,0.445567,0.373631,0.441707,0.441707,10526649978
2021-05-04,0.440274,0.605998,0.400800,0.541334,0.541334,38238126181


In [35]:
#set index as date 
#df = df.set_index(pd.DatetimeIndex(df['Date'].values))

KeyError: 'Date'

In [42]:
df.groupby(df.index.year).Volume.sum().reset_index()

,Date,Volume
0,2019,69469785
1,2020,46791627016
2,2021,713285539731


In [55]:
df.index

DatetimeIndex(['2019-12-31', '2020-01-01', '2020-01-02', '2020-01-03',
               '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07',
               '2020-01-08', '2020-01-09',
               ...
               '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30',
               '2021-05-01', '2021-05-02', '2021-05-03', '2021-05-04',
               '2021-05-05', '2021-05-06'],
              dtype='datetime64[ns]', name='Date', length=489, freq=None)

In [56]:
df = df[['Close']]

In [109]:
prediction_days = 2

In [58]:
df

,Close
Date,
2019-12-31,0.002028
2020-01-01,0.002033
2020-01-02,0.002009
2020-01-03,0.002145
2020-01-04,0.002241
...,...
2021-05-02,0.376046
2021-05-03,0.441707
2021-05-04,0.541334


In [59]:
df['prediction'] = df[['Close']].shift(-prediction_days)

<ipython-input-59-0c6df9d41771>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction'] = df[['Close']].shift(-prediction_days)


In [60]:
df

,Close,prediction
Date,,
2019-12-31,0.002028,0.002033
2020-01-01,0.002033,0.002009
2020-01-02,0.002009,0.002145
2020-01-03,0.002145,0.002241
2020-01-04,0.002241,0.002419
...,...,...
2021-05-02,0.376046,0.441707
2021-05-03,0.441707,0.541334
2021-05-04,0.541334,0.657633


In [62]:
#create independent value X
X = np.array(df.drop(['prediction'],1))
#remove last n+1 row 
X = X[:len(df)-prediction_days-1]

In [64]:
#dependent row 
y= np.array(df['prediction'])
#delete last column
y = y[:-prediction_days-1]

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [98]:
from sklearn.ensemble import RandomForestRegressor
forest =  RandomForestRegressor(n_estimators=2,random_state = 285 )
forest.fit(X_train,y_train)
print(forest.score(X_test,y_test))

0.9121435771614516


In [101]:
prediction =  forest.predict(X_test)
#predicted test value
print(prediction)
print()
#predicted values
print(y_test)

[0.002627   0.002809   0.0032765  0.0032045  0.009171   0.004538
 0.0591515  0.0605645  0.0503375  0.082106   0.0027735  0.003079
 0.002447   0.007482   0.082106   0.0026225  0.003295   0.002409
 0.002313   0.003256   0.002599   0.0025465  0.002627   0.00258
 0.002775   0.002758   0.002533   0.001812   0.00194    0.0028115
 0.051448   0.00267375 0.00204    0.049376   0.057998   0.057383
 0.003442   0.002266   0.002532   0.00258    0.002603   0.38451651
 0.00259    0.082106   0.00251775 0.002052   0.002599   0.003274
 0.002551   0.002513   0.002447   0.00227271 0.001812   0.004636
 0.002599   0.054052   0.002583   0.417347   0.002855   0.00251775
 0.003329   0.003446   0.002052   0.003533   0.0028515  0.002313
 0.002463   0.0024805  0.003295   0.06645    0.00316287 0.00204
 0.003442   0.003303   0.002581   0.002381   0.002787   0.00204
 0.002967   0.002266   0.0024045  0.407318   0.0022785  0.0018145
 0.003811   0.002613   0.0028515  0.002394   0.0015765  0.0035505
 0.0536225  0.002447 

In [107]:
temp_df = df[:-prediction_days]
#to store independent price values
x_val = temp_df.tail(1)['Close'][0]
x_val

0.6576330065727234

In [108]:
prediction = forest.predict([[x_val]])
#price for next n days
print('price of doge coin in', prediction_days, 'day(s) is predicted to be', prediction )
#print the actual value
print('actual price', temp_df.tail(1)['prediction'][0])

price of doge coin in 1 day(s) is predicted to be [0.38451651]
actual price 0.5808039903640747
